# Latent Semantic Analysis using nouns and noun phrases from <code>ds_PWDB</code> dataset

Text data suffers heavily from high-dimensionality. Latent Semantic Analysis (LSA) is a popular,
dimensionality-reduction techniques that follows the same method as Singular Value Decomposition.
LSA ultimately reformulates text data in terms of r <b>latent</b> (i.e. <b>hidden</b>) features, where r is less than m,
the number of terms in the data. We will use <code>ds_PWDB</code> textual data to preform the LDA model.

### Import the libraries

In [1]:
from typing import List
import warnings
warnings.filterwarnings('ignore')

import spacy
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sem_covid.services.data_registry import Dataset
from sem_covid.entrypoints.notebooks.topic_modeling.topic_modeling_wrangling.topic_visualizer import \
    plotly_bar_chart_graphic
from sem_covid.services.sc_wrangling.data_cleaning import clean_text_from_specific_characters, clean_remove_stopwords
from sem_covid.entrypoints.notebooks.topic_modeling.topic_modeling_wrangling.tfidf_vectorization import vectorize_documents
from sem_covid.entrypoints.notebooks.topic_modeling.topic_modeling_wrangling.lsa_transformer import lsa_document_transformation

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning:

`scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


### Download the dataset

In [2]:
pwdb = Dataset.PWDB.fetch()
pwdb.head()

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

100% (1288 of 1288) |####################| Elapsed Time: 0:00:01 Time:  0:00:01


,identifier,title,title_national_language,country,start_date,end_date,date_type,type_of_measure,status_of_regulation,category,...,funding,involvement_of_social_partners_description,social_partner_involvement_form,social_partner_role,is_sector_specific,private_or_public_sector,is_occupation_specific,sectors,occupations,sources
_id,,,,,,,,,,,,,,,,,,,,,
adc5c75937bc7f7198f534d08b85bd50c9521bfd3f319a090932b5d0bae54de0,1297,Agreement on a teleworking regime,Convention du 20 octobre 2020 relative au régi...,Luxembourg,10/20/2020,None,Open ended,Bipartite collective agreements,Entirely new measure,"Protection of workers, adaptation of workplace",...,[Companies],The agreement is a social partner initiative w...,None,None,No,Only private sector,No,[],[],[{'title': 'Accord signé entre partenaires soc...
2372d71eb9ad6e6a70982e02bbe802db004ed49d91b2264c0a2e8e41571002cc,864,Special protection for COVID-19 risk groups at...,Besonderer Schutz von Risikogruppen,Austria,05/06/2020,05/31/2021,Temporary,Legislations or other statutory regulations,Entirely new measure,"Protection of workers, adaptation of workplace",...,[National funds],The expert group which elaborated the definiti...,None,None,No,Not specified,No,[],[],"[{'title': 'FAQs Risk groups', 'url': 'https:/..."
8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353,1228,Funds for innovative renewable projects in And...,Ayudas para proyectos de renovables en Andaluc...,Spain,08/25/2020,None,Open ended,Legislations or other statutory regulations,Entirely new measure,"Promoting the economic, labour market and soci...",...,"[Employer, European Funds, Local funds, Nation...",No involvement reported,None,None,Yes,Not specified,No,"[Electricity, gas, steam and air conditioning ...",[],[{'title': 'LÍNEAS DE AYUDA PARA PROYECTOS REN...
18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044,183,Waiver of advance payments for social and heal...,Odklad záloh sociálního a zdravotního pojištěn...,Czechia,03/01/2020,08/31/2020,Temporary,Legislations or other statutory regulations,Entirely new measure,Supporting businesses to stay afloat,...,[No special funding required],"Social partners, who are members of the tripar...",None,None,No,Not specified,No,[],[],[{'title': 'CSSZ: Self-employed persons - coro...
b94d8aa95fbdeb1bb832b01fbe5d6e9bf9fc36fceb14f7ba370a963f472fe35b,1550,Financial Shield 2.0: Small and medium-sized e...,Tarcza Finansowa 2.0: małe i średnie przedsięb...,Poland,01/01/2021,None,Temporary,Legislations or other statutory regulations,New aspects included into existing measure,Supporting businesses to stay afloat,...,[National funds],None.,None,None,Yes,Only private sector,No,"[Manufacture of paper and paper products, Prin...",[],[{'title': 'The Act of 31 March 2020 amending ...


### Prepare the data
After importing and visualize the dataset table, we detected the columns that contains textual data and concatenate
them to have an entire text to wotk with.

In [3]:
pwdb_descriptive_data = pwdb['title'].map(str) + ' ' + \
                        pwdb['background_info_description'].map(str) + ' ' + \
                        pwdb['content_of_measure_description'].map(str) + ' ' + \
                        pwdb['use_of_measure_description'] + ' ' + \
                        pwdb['involvement_of_social_partners_description']

pwdb_descriptive_data

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



_id
adc5c75937bc7f7198f534d08b85bd50c9521bfd3f319a090932b5d0bae54de0    Agreement on a teleworking regime During the C...
2372d71eb9ad6e6a70982e02bbe802db004ed49d91b2264c0a2e8e41571002cc    Special protection for COVID-19 risk groups at...
8735e268191e9e5cbd3d2a44ca53d297e31746b5f1e24b941db6225a25848353    Funds for innovative renewable projects in And...
18bcd22116c46919e03a3345f793c3859855227ac942e69dd13cbfcd588e1044    Waiver of advance payments for social and heal...
b94d8aa95fbdeb1bb832b01fbe5d6e9bf9fc36fceb14f7ba370a963f472fe35b    Financial Shield 2.0: Small and medium-sized e...
                                                                                          ...                        
cb014a456b14c3621dd318a12e611f70c2a9636be9fe181072bd4bf5917a40fa    Automatic extension of unemployment benefits D...
d233b17dc2b98f14269c2b22be78d93ec5ccf2a0013b86f09175c69353c5800b    Extra subsidies to institutions in the cultura...
77d7e3c52aaf78bdfb1a1667641db1293bbff862440c547fc3f6

### Clean the data

After we extract the data, one of the most important step is cleaning them. We will delete below characters
from our text to make it cleaner.

In [4]:
unused_characters = ["\\r", ">", "\n", "\\", "<", "''", "%", "...", "\'", '"', "(", "\n"]
clean_text = clean_text_from_specific_characters(pwdb_descriptive_data, unused_characters)
clean_text

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



'[Agreement teleworking regime During COVID-19 crisis, teleworking identified vital pillar companies working prevent social hardship. Discussions representative social partners OGBL LCGB employer association UEL, Ministry Work Employment, led instance joint assessment teleworking level Economic Social Council CES. From there, discussions continued social partners inter-professional agreement signed social partners October 2020. Applied period years covering sectors Luxembourg with exception transport), agreement provides definition teleworking: * Teleworking identified form organisational work, conducted digital means usually company, transferred location employee lives. * The work considered teleworking applied occasional exceptional circumstances, remains 10 threshold annual working time. * Teleworking based written agreement employer employee, containing compulsory elements, example location telework takes place number hours, employee cant dismissed he/she accept teleworking scheme 

### Extract nouns and noun phrases

In [5]:
doc = nlp(clean_text)

nouns = [token for token in doc if token.pos_ == "NOUN"]
str_nouns = [str(noun) for noun in nouns]
noun_phrases = [word.text for word in doc.noun_chunks]

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



### TFIDF Training

Our models work on numbers, not string! So we tokenise the text (turning all documents into smaller observational
entities — in this case words) and then turn them into numbers using Sklearn’s TF-IDF vectoriser.
This should gives us our vectorised text data — the document-term matrix.

Now let’s visualise the singular values

In [7]:
noun_phrases_vectors = vectorize_documents(noun_phrases)
noun_phrases_vectors

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,19,agreement,covid,employer,group,measure,partners,representatives,risk,security,social,wage,work,workers,working
0,0.707107,0.0,0.707107,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.0,0.76067,0.0,0.0,0.0,0.649139,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,1.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
288,0.000000,0.0,0.000000,0.0,0.0,0.0,0.76067,0.0,0.0,0.0,0.649139,0.0,0.0,0.0,0.0
289,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
290,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [8]:
noun_vectors = vectorize_documents(str_nouns)
noun_vectors

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,19,covid,risk,wage
0,0.000000,0.000000,0.0,0.0
1,0.000000,0.000000,0.0,0.0
2,0.707107,0.707107,0.0,0.0
3,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.0,0.0
...,...,...,...,...
526,0.000000,0.000000,0.0,0.0
527,0.000000,0.000000,0.0,0.0
528,0.000000,0.000000,0.0,0.0
529,0.000000,0.000000,0.0,1.0


### LDA Modeling

Let’s explore our reduced data through the term-topic matrix, V-tranpose. TruncatedSVD will return it to as a numpy
array of shape (num_documents, num_components), so we’ll turn it into a Pandas dataframe for ease of manipulation.

Let’s slice our term-topic matrix into Pandas Series (single column data-frames), sort them by value and plot them.
The code below plots this for our 2nd latent component (recall that in python we start counting from 0)
and returns the plot.

In [17]:
noun_phrases_lsa = lsa_document_transformation(noun_phrases_vectors, num_components=14)
noun_lsa = lsa_document_transformation(noun_vectors, num_components=3)

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [18]:
singular_values_noun_phrases_plot = plotly_bar_chart_graphic("Singular values", list(range(len(noun_phrases_lsa[1]))),
                                                noun_phrases_lsa[1], "Latent component",
                                                "Relative importance of each component")
singular_values_noun_phrases_plot

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [19]:
singular_values_noun_plot = plotly_bar_chart_graphic("Singular values", list(range(len(noun_lsa[1]))),
                                                noun_lsa[1], "Latent component", "Relative importance of each component")

singular_values_noun_plot

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [20]:
noun_phrases_term_topic_matrix = pd.DataFrame(data=noun_phrases_lsa[2], index=noun_phrases_vectors.columns,
                                              columns= [f'Latent_concept_{r}' for r in range(0 , noun_phrases_lsa[2].shape[1])])
noun_phrases_term_topic_matrix

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,Latent_concept_0,Latent_concept_1,Latent_concept_2,Latent_concept_3,Latent_concept_4,Latent_concept_5,Latent_concept_6,Latent_concept_7,Latent_concept_8,Latent_concept_9,Latent_concept_10,Latent_concept_11,Latent_concept_12,Latent_concept_13
19,4.923897e-01,-1.010477e-01,-5.398702e-02,-1.735063e-01,1.016837e-14,3.858969e-01,2.556504e-01,1.005462e-04,1.721313e-03,-6.904004e-04,-1.081856e-04,6.740114e-03,5.142945e-04,8.476732e-05
agreement,8.204981e-03,4.802025e-02,-2.269724e-02,2.030607e-04,-4.217989e-16,-7.075427e-04,-5.988940e-04,2.631373e-01,7.996132e-06,3.958474e-02,9.500491e-01,3.363310e-02,-1.341876e-01,-6.761446e-02
covid,4.923897e-01,-1.010477e-01,-5.398702e-02,-1.735063e-01,1.017527e-14,3.858969e-01,2.556504e-01,1.005462e-04,1.721313e-03,-6.904004e-04,-1.081856e-04,6.740114e-03,5.142945e-04,8.476732e-05
employer,7.187437e-04,4.375693e-03,-3.249745e-03,4.779925e-05,-1.683645e-16,-2.539946e-04,-3.279844e-04,9.644138e-01,-3.809827e-06,-1.452237e-02,-2.625968e-01,-6.792089e-03,2.487752e-02,6.537126e-03
group,2.513633e-01,-2.762242e-02,6.994063e-02,3.383462e-02,-8.122948e-15,-1.902549e-01,-1.943600e-01,2.232640e-04,-6.529118e-03,-1.293202e-02,-3.729320e-03,9.043882e-01,1.946383e-01,-4.296261e-03
measure,-1.466926e-17,1.905981e-16,-1.804371e-16,4.241789e-16,1.000000e+00,5.935478e-15,-4.845377e-14,2.223446e-16,2.332639e-16,1.679987e-17,5.762426e-16,-5.235713e-17,-1.344878e-16,-1.444689e-16
partners,8.025754e-02,4.713412e-01,-2.328232e-01,2.197204e-03,-5.783557e-16,-7.965200e-03,-6.872390e-03,-2.430171e-02,-3.176644e-04,-3.461055e-01,-1.295514e-01,1.251320e-01,-5.854021e-01,-4.706985e-01
representatives,1.698057e-02,9.834091e-02,-4.001974e-02,2.780077e-04,-1.840407e-16,-6.914363e-04,-3.367996e-04,-3.948241e-03,2.528327e-04,9.117134e-01,-1.011121e-01,9.325484e-02,-3.701076e-01,-3.607855e-02
risk,6.376759e-01,-1.242786e-01,-2.316718e-02,1.599615e-01,-1.628271e-14,-5.084377e-01,-3.964431e-01,-3.203205e-04,-3.080328e-03,7.115194e-03,1.775775e-03,-3.613745e-01,-7.394471e-02,1.071398e-03
security,4.511632e-02,2.363026e-01,6.417759e-03,-7.887824e-04,1.145749e-16,4.138740e-03,4.234286e-03,-4.208470e-03,2.592846e-04,2.093534e-01,2.045972e-02,-1.355164e-01,6.096396e-01,-7.125717e-01


In [21]:
noun_term_topic_matrix = pd.DataFrame(data=noun_lsa[2], index=noun_vectors.columns,
                                              columns= [f'Latent_concept_{r}' for r in range(0 , noun_lsa[2].shape[1])])
noun_term_topic_matrix

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,Latent_concept_0,Latent_concept_1,Latent_concept_2
19,0.0,7.071068e-01,-1.003637e-22
covid,0.0,7.071068e-01,-1.003637e-22
risk,1.0,0.000000e+00,-0.000000e+00
wage,0.0,1.419357e-22,1.000000e+00


In [22]:
class TermTopicMatrix(object):
    def __init__(self, matrix: pd.DataFrame, num_concepts: int):
        self.matrix = matrix
        self.num_concepts = num_concepts

    def top_term_topic(self):
        for number in range(self.num_concepts):
            data = self.matrix[f'Latent_concept_{number}']
            data = data.sort_values(ascending=False)
            top_10 = data[:10]

            plotly_bar_chart_graphic(f"Top term along the axis of latent concept {number}",
                                 top_10.index, top_10.values, "Noun phrases", "LSA Term Topic")

    def bottom_term_topic(self):
        for number in range(self.num_concepts):
            data = self.matrix[f'Latent_concept_{number}']
            data = data.sort_values(ascending=False)
            bottom_10 = data[10:]

            plotly_bar_chart_graphic(f"Bottom term along the axis of latent concept {number}",
                                 bottom_10.index, bottom_10.values, "Noun phrases", "LSA Term Topic")

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [25]:
noun_phrases_topic_matrix = TermTopicMatrix(noun_phrases_term_topic_matrix, 14)
noun_topic_matrix = TermTopicMatrix(noun_term_topic_matrix, 3)

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



### Top topics from noun phrases

In [26]:
noun_phrases_top_topic = noun_phrases_topic_matrix.top_term_topic()

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



### Bottom topics from noun phrases

In [27]:
noun_phrases_bottom_topic = noun_phrases_topic_matrix.bottom_term_topic()

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



### Top topics from nouns

In [28]:
noun_top_topic = noun_topic_matrix.top_term_topic()

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



### Bottom topics from nouns

In [29]:
noun_bottom_topic = noun_topic_matrix.bottom_term_topic()

/home/daycu/Work/sem-covid/venv/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

